## Geneious analysis for individual samples from raw Geneious output, "Annotation.csv" 

 ### Required packages
 - No specific package required 
 
 ### Inputs
 - Geneious SNP analysis of _k13_, _crt_, _mdr1_, _dhfr_, _dhps_, and _cytb_
 - Documentation on Geneious analysis can be found: Readme.md
 - Geneious outputs were modified to GuineaAnalysis_Individual.csv from "Annotation.csv"
 
 
 ### Data structure 
 - [Long-form](https://seaborn.pydata.org/tutorial/data_structure.html#long-form-vs-wide-form-data) 
     - Each variable is a column 

         - "Sample" = *AMD ID*, including associated meta-data for each sample
             - AMD ID and bit code key is found under MS Teams > Domestic > Files > Sample Naming > Sample_naming_key.pptx  

             - Key: **Year Country State/Site DayofTreatment Treatment SampleID Genus SampleType GeneMarker-8bitcode SampleSeqCount**

                 - Example:
                     - Individual sequenced sample ID: 17GNDo00F0001PfF1290 = 2017 Guinea Dorota Day0 AS+AQ 0001 P.falciparum FilterBloodSpot k13-crt-mdr-dhfr-dhps-cytB-cpmp-pfs47 

                     - Pooled sequenced sample ID: 17GNDoxxx001P10F1290 = 2017 Guinea Dorota **xx x** 001 **Pooled SamplesInPool** P.falciparum FilterBloodSpot k13-crt-mdr-dhfr-dhps-cytB-cpmp-pfs47 

                         - NOTE: If information is not availble (na) **x** is used. For pooled samples, DayofTreatment and Treatment is na since its a pool of multiple samples with that info. 
                         - NOTE: For pooled samples, **Genus** is replaced with **Pooled** and **SampleType** with **SamplesInPool** to indicated this as a pooled sequenced sample and sample count in each pool. 
         <p>&nbsp;</p>
         - "Year" = the year the study was conducted 
         - "Site" = the state or province 
         - "Day_of_treatment" = describes the day of treatment provided to the patient 
         - "Gene" = drug resistant gene(s) 
         - "G_annotation" = full SNP annotation in the following format: WildTypeAA-CodonPosition-MutantAA 
         - "Coverage" = the number of reads covering the SNP 
         - "VAF" = variant allele frequency calculated by AA divided by total reads in loci 
         - "SNP" = single nucleotide polymorphism in WildTypeAA or MutantAA annotation format 
         - "Type" = describes if it is a wild type or mutant SNP 

     - Each observation is a row for each sample ID (patient ID) 
 
 #### TODO
 
 #### Activity Name
 - [ ] Write doc.string at the beginning of the code
 - [ ] Write detailed description with comment for line by line
 - [ ] Make the code more simple and accurate
 - [ ] Follow zen of python
    
 #### Completed Activity ✓
 - [x] Created marked down at the beginning of the file for description

In [88]:
import pandas as pd ## Import Pandas library for processing dataframe as pd
import numpy as np ## Import Numy for processing matrix as np
Geneious_DF=pd.read_csv("Annotations.csv") ##Import raw Geneious output for variant analysis
Geneious_DF_N1=Geneious_DF[(Geneious_DF['Type']=='Polymorphism') & (Geneious_DF['Amino Acid Change'].notnull())] ##If the type column contains polymorphism and Amino Acid Change column is not empty then create a dataframe satisfying those conditions

Geneious_DF_N2=Geneious_DF[Geneious_DF['Type']=='Coverage - High'] ##If the Coverage - High is in the type column as value then select dataframe for those column values

Geneious_DF_N1["TrackerSNP"]=Geneious_DF_N1["Amino Acid Change"].astype(str).str[0]+Geneious_DF_N1["CDS Codon Number"].astype(int).astype(str)+Geneious_DF_N1["Amino Acid Change"].astype(str).str[-1] ##Create a TrackerSNP Column which has both amino acid before the change and after the change

Combine_Variant_Wildtpye = [Geneious_DF_N1, Geneious_DF_N2] ##Produce a complete dataframe which contains both variants and wildtypes
Combation_Vi_Wi = pd.concat(Combine_Variant_Wildtpye) ##Concatenate the dataframes for variants and wildtypes
Combination_filtered=Combation_Vi_Wi.drop_duplicates(subset =["Document Name", "TrackerSNP"] )  ##Drop duplicates meaning if the values are already in variants then drop it from the wildtypes
##The dataframe contains information, "Sample,Pooled,Year,SITE,TreatmentDay,GENE,G_annotation,COVERAGE,VAF,VF,SNP,TYPE\n")
        
def site(row): ##Set up a function for assignging site based on the values in the document name column
    if row['Document Name'][4:6]=="Ha":
        return 'Hamdalaye'
    elif row['Document Name'][4:6]=="Do":
        return 'Dorota'
    elif row['Document Name'][4:6]=="Ma":
        return 'Maferinyah'
    elif row['Document Name'][4:6]=="La":
        return 'Lay-Sare'
    elif row['Document Name'][4:6]=="LS":
        return 'Lay-Sare'
    
def TreatmentDay(row): ##Set up a function for assigning TreatmentDay based on the values in the document name column
    if row['Document Name'][6:8]=="00":
        return '0'
    elif row['Document Name'][6:8]=="1A":
        return '1'
    elif row['Document Name'][6:8]!="00" and row['Document Name'][6:8]!="1A":
        return row['Document Name'][6:8]
    
def Pooled(row): ##Set up a function for Pooled based on the values in the document name column
    if row['Document Name'][8:10]=="xp":
        return 'pooled'
    elif row['Document Name'][8:10]!="xp":
        return 'individual'

def year(row):  ##Set up a function for Year  based on the values in the document name column
    return row['Document Name'][0:2]

def type(row):  ##Set up a TYPE column based on given value in the Type whether it is mutation or wildtype
    if row['Type'] =='Polymorphism':
        return "mutation"
    if row['Type'] =='Coverage - High':
        return "wildtype"
    
def SNP(row):  ##Set up a SNP column to give pre or post amino acid changes based on mutation or wildtype
    if row['Type'] =='Polymorphism':
        return row['TrackerSNP'][1::]
    if row['Type'] =='Coverage - High':
        return row['TrackerSNP'][0:-1]
    
    


    
Combination_filtered["SITE"]=Combination_filtered.apply(site, axis=1) ##Apply the functions defined previously
Combination_filtered["TreatmentDay"]=Combination_filtered.apply(TreatmentDay, axis=1)
Combination_filtered["Pooled"]=Combination_filtered.apply(Pooled, axis=1)
Combination_filtered["Year"]=Combination_filtered.apply(year, axis=1)
Combination_filtered["TYPE"]=Combination_filtered.apply(type, axis=1)
Combination_filtered["SNP"]=Combination_filtered.apply(SNP, axis=1)



Combination_report1=Combination_filtered[Combination_filtered['Type']=='Polymorphism'] ##Select columns with mutations
Combination_report2=Combination_filtered[Combination_filtered['Type']=='Coverage - High'] ##Select columns with wildtypes
final_report1=Combination_report1[["Document Name","Sequence Name","SITE","TreatmentDay","Pooled","Year","Coverage","Variant Frequency","Variant Raw Frequency","TrackerSNP","TYPE","SNP"]] ##Assign sample information to samples with mutation
final_report2=Combination_report2[["Document Name","Sequence Name","SITE","TreatmentDay","Pooled","Year","Average Coverage","Variant Frequency","Variant Raw Frequency","TrackerSNP","TYPE","SNP"]] ##Assign sample information to samples with wildtypes
final_report2_re=final_report2.rename(columns={'Average Coverage': 'Coverage'}) ##Change the name of average coverage to coverage for samples with wildtypes
final_combine=[final_report1, final_report2_re] ##Combine the information from wildtypes and mutations into one dataframe
final_combine_2=pd.concat(final_combine) ##concatenate
final_combine_2.to_csv("test.csv", sep=',', index=False) ##Create a file with the dataframe for testing purpose



/tmp/ipykernel_117/2388402328.py:3: DtypeWarning: Columns (16,19,24,26,28,30,31,32,33,34,35,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,62,64,65,66,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  Geneious_DF=pd.read_csv("Annotations.csv") ##Import raw Geneious output for variant analysis
/tmp/ipykernel_117/2388402328.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Geneious_DF_N1["TrackerSNP"]=Geneious_DF_N1["Amino Acid Change"].astype(str).str[0]+Geneious_DF_N1["CDS Codon Number"].astype(int).astype(str)+Geneious_DF_N1["Amino Acid Change"].astype(str).str[-1] ##Create a TrackerSNP Column which has both amino acid before the change and after the change
/tmp/ipykernel_117/2388402328.py:60: S

In [89]:
pooled_part1=pd.read_csv("Pooled_Info_Part1.csv") ##import a file with information about pooled samples
pooled_part2=pd.read_csv("Pooled_Info_Part2_fixed.csv")  ##import a file with information about pooled samples

Combine_pooled_parts = [pooled_part1[["Pool","SITE","YEAR","AMD_ID","Poolsize"]], pooled_part2[["Pool","SITE","YEAR","AMD_ID","Poolsize"]]]
Combation_pooled_concatenate = pd.concat(Combine_pooled_parts) ##Combine and concatenate the two pooled files

Combation_pooled_concatenate.to_csv("test-pre1.csv", sep=',', index=False) ##create file for testing purpose

def name(row):
    return row['Document Name'].split("_")[0]

final_combine_2["Document Name"]=final_combine_2.apply(name, axis=1) ##Clean the document name which is the AMD_ID to get rid of the Geneious information

final_combine_2.rename(columns={'Document Name':'AMD_ID'}, inplace=True) ##Reanme the document name to AMD_ID


df_merged_poolsize = pd.merge(final_combine_2, Combation_pooled_concatenate, on=['AMD_ID'], how='left') ##merge the information based on AMD_ID to add pooled columns and pooled size columns

df_merged_poolsize = df_merged_poolsize.drop('SITE_y', 1) ##Get rid of duplicate columns which is SITE_y
df_merged_poolsize = df_merged_poolsize.drop('YEAR', 1) ##Get rid of duplicate columns which is year

df_merged_poolsize.Poolsize.fillna(value=1, inplace=True) ##Fill empty values for poolsize for individual with 1s

df_merged_poolsize['Variant Frequency'] = df_merged_poolsize['Variant Frequency'].fillna(0)

print(df_merged_poolsize["Variant Frequency"].str.split('%').str[0].str.strip("%"))

df_merged_poolsize["Prod"]=df_merged_poolsize["Variant Frequency"].str.split('%').str[0].str.strip("%").astype(float)*df_merged_poolsize["Poolsize"].astype(float)

df_merged_poolsize.to_csv("test2.csv", sep=',', index=False) ##Generate file to test


 

0        100.0
1        100.0
2        100.0
3        100.0
4        100.0
         ...  
11444      NaN
11445      NaN
11446      NaN
11447      NaN
11448      NaN
Name: Variant Frequency, Length: 11449, dtype: object


/tmp/ipykernel_117/1774320522.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_merged_poolsize = df_merged_poolsize.drop('SITE_y', 1) ##Get rid of duplicate columns which is SITE_y
/tmp/ipykernel_117/1774320522.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_merged_poolsize = df_merged_poolsize.drop('YEAR', 1) ##Get rid of duplicate columns which is year


PermissionError: [Errno 13] Permission denied: 'test2.csv'

In [110]:
df_merged_count=df_merged_poolsize.groupby(['SITE_x','TrackerSNP', 'Pooled']).sum()  ##Sum the columns based on overlapping values on site, trackersnp, and pooled
df_merged_countv=df_merged_poolsize.groupby(['SITE_x','TrackerSNP', 'Pooled', 'SNP']).sum() ##Sum the columns based on agreeing values on site, trackersnp, and pooled, and snp
df_merged_count=df_merged_count.groupby(['SITE_x','TrackerSNP']).sum() ##Sum again based on stie and tracker snp
df_merged_countv=df_merged_countv.groupby(['SITE_x','TrackerSNP','SNP']).sum() ##Sum again based on site, tracker, and snp
df_merged_countv = df_merged_countv.drop('Pool', 1) ##drop pooled information

#df_merged_countv.to_csv("testtest1.csv", sep=',', index=True)

df_merged_countv=df_merged_countv.reset_index()  
df_merged_countv['Type'] = np.where(df_merged_countv['SNP'].str[0].str.isdigit(), "Mutation" , "WildType")
df_merged_countv.rename(columns={'Poolsize':'Number_of_samples'}, inplace=True)
df_merged_countvp=df_merged_countv
df_merged_countv=df_merged_countv.pivot(index=["SITE_x", "TrackerSNP"], columns="Type", values="Number_of_samples") ##pivot and align mutation and wildtype
df_merged_countvp=df_merged_countvp.pivot(index=["SITE_x", "TrackerSNP"], columns="Type", values="Prod") ##pivot and align mutation and wildtype

#df_merged_countvp.to_csv("testtest1.csv", sep=',', index=True)

"""
To do: 
https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
This is where I am stuck. Trying using pivot to fix it
"""

df_merged_countvp=df_merged_countvp.reset_index()
df_merged_countv=df_merged_countv.reset_index()  

df_merged_countv['SNP'] = np.where(pd.isna(df_merged_countv['Mutation']), df_merged_countv['TrackerSNP'].astype(str).str[0:-1], df_merged_countv['TrackerSNP'].astype(str).str[1::])

cols = list(df_merged_countv.columns)
cols = cols[0:2] + cols[4:5] + cols[2:4]
df_merged_countv = df_merged_countv[cols]
df_merged_countv.rename(columns={'TrackerSNP':'G_Annotation'}, inplace=True)

df_merged_countv['Prod']= df_merged_countvp['Mutation']

df_merged_countv['Prod'] = df_merged_countv['Prod'].fillna(0)

df_merged_countv['Mutation'] = df_merged_countv['Mutation'].fillna(0)

df_merged_countv['WildType'] = df_merged_countv['WildType'].fillna(0)

df_merged_countv['sum'] = df_merged_countv['Mutation'] + df_merged_countv['WildType']

df_merged_countv['div'] = df_merged_countv['Prod'] / df_merged_countv['sum']

df_merged_countv['div']=df_merged_countv['div'].round(2)

df_merged_countv['div']=np.where(pd.isna(df_merged_countv['div']),"", df_merged_countv['div'].astype(str)+"%") 
#df_merged_countv['div'].astype(str)+"%"

df_merged_countv = df_merged_countv.drop('Prod', 1) ##drop pooled information
df_merged_countv = df_merged_countv.drop('sum', 1) ##drop pooled information

df_merged_countv.rename(columns={'div':'VAF'}, inplace=True)



df_merged_countv.to_csv("testtest3.csv", sep=',', index=True)
#df_merged_countivsr=df_merged_countivs.reset_index()
#df_merged_countivsr2=df_merged_countivs.reset_index()
#df_merged_countivsr.drop(df_merged_countivsr.index[df_merged_countivsr['SNP'].str[0].str.isdigit()], inplace=True)
#df_merged_countivsr2.drop(df_merged_countivsr.index[df_merged_countivsr['SNP'].str[0].str.isdigit()==False], inplace=True)
#df_merged_countivsr2.to_csv("test3.csv", sep=',', index=True)
#df_merged_countivsr.to_csv("test4.csv", sep=',', index=True)
#df_merged_countivsr_comb = pd.merge(df_merged_count, df_merged_countv, on=['SITE_x','TrackerSNP'], how='left')
#df_merged_countivsr_comb = df_merged_countivsr_comb.drop('Pool_x', 1)
#df_merged_countivsr_comb = df_merged_countivsr_comb.drop('Pool_y', 1)
#df_merged_countivsr_comb.to_csv("test5.csv", sep=',', index=True)



df_merged_countis.rename(columns={'SITE_x':'SITE'}, inplace=True)
#df_merged_countivsr.to_csv("test3.csv", sep=',', index=True)
df_merged_countpool = pd.merge(df_merged_counti, Combation_pooled_concatenate, on=['SITE','TrackerSNP','Pooled'], how='left') 
#df_merged_countpool.to_csv("test3.csv", sep=',', index=False)
df_merged_countpool.Poolsize_y.fillna(value=1, inplace=True)
df_merged_countpool_filtered = df_merged_countpool[['AMD_ID','Sequence Name','Poolsize_y']]
df_merged_countpool_filtered.rename(columns={'Sequence Name':'Sequence_Name'}, inplace=True)
df_merged_countpool_filtered.to_csv("test4.csv", sep=',', index=False)
values = df_merged_countpool_filtered.Sequence_Name * df_merged_countpool_filtered.Poolsize_y
df_merged_countpool_filtered['values']=values
#print(df_merged_countpool_filtered)
df_merged_countpool_filtered.to_csv("test5.csv", sep=',', index=False)

/tmp/ipykernel_117/3914089270.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_merged_countv = df_merged_countv.drop('Pool', 1) ##drop pooled information
/tmp/ipykernel_117/3914089270.py:51: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_merged_countv = df_merged_countv.drop('Prod', 1) ##drop pooled information
/tmp/ipykernel_117/3914089270.py:52: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_merged_countv = df_merged_countv.drop('sum', 1) ##drop pooled information


NameError: name 'df_merged_countis' is not defined

In [ ]:
df_merged_poolsize[]